In [ ]:
#code to find the python version
import platform 
print(platform.python_version())

In [ ]:
import json
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
from sklearn.linear_model import LinearRegression

## Examining the file

Before loading the .dat file, it was opened and examined in Visual code to have a look at the contents and format of the file. We found that the file consisted of {,[ which is very likely to be JSON.

In [ ]:
#Parsing the data.dat file
with open("data.dat") as json_file:
    json_data = json.load(json_file)
print type(json_data)
#json_data['meta']['view']

In [ ]:
#Viewing the first record of the file.
json_data['houses'][0]

In [ ]:
#Converting the JSON object into the data frame usong the json_normalize
df = json_normalize(json_data['houses'])

In [ ]:
#looking into the normalize data
df.head()

### The dataframe __`df`__ above has following issues that must be fixed to meet the requirement of the task:
1. `address` is a composite attribute that needs to be sub-divied into street, city, statezip, and country.
2. `rooms` is a composite attribute that needs to be sub-divided into bedrooms and bathrooms. 
3. `sqft_living` and `sqft_lot` are in one column. It must be separated into 2 columns.
4. Check and change data types of columns where required.

#### Spliting the address column into street, city , statezip and country

In [ ]:
# ref :https://stackoverflow.com/questions/14745022/pandas-dataframe-how-do-i-split-a-column-into-two ;, by LeoRochel
# the following code splits the column into 4  columns and add the new columns on the dataframe
df = df.join(df['address'].str.split(',', 3, expand=True).rename(columns={0:'street', 1:'city', 2:'statezip', 3:'country'}))




In [ ]:
df.head()

The above command has worked perfectly and the address has been split into different columns and added on to the dataframe

#### Splitting the room column
The below code will split the room column into bedrooms and bathrooms and add the new column into the dataframe.

In [ ]:
#https://stackoverflow.com/questions/14745022/pandas-dataframe-how-do-i-split-a-column-into-two ;, by LeoRochel

df = df.join(df['rooms'].str.split(';', 2, expand=True).rename(columns={0:'bathrooms', 1:'bedrooms'}))




In [ ]:
df.head()

The above command has worked perfectly and the new columns bathrooms and bedrooms has been added.

#### Checking the consistency of the newly added columns

The following code checks for the newly added column bathrooms, if all the values starts with "Number of bathrooms " and no value starts with "Number of bedrooms"

In [ ]:
df['checkbath'] = df['bathrooms'].apply(lambda value: value.startswith('Number of bedrooms'))
len(df[df["checkbath"] == True])

The above command adds a new column in the checkbath and stores the boolean value. It is seen that there are 400 rows that contains the bedrooms value in the bathroom column. Lets check the same for the bedrooms

In [ ]:
df['checkbed'] = df['bedrooms'].apply(lambda value: value.startswith(' Number of bathrooms'))
len(df[df["checkbed"] == True])

The bedrooms column contians 400 values of the bathrooms. Now lets check if all these values for bedrooms and bathroom have been reversly added.

In [ ]:
#the following code gets the index of all the bathrooms that have bedrooms value
idx_checkbath = (df[df["checkbath"] == True]).index.tolist()

len(idx_checkbath)

In [ ]:
#the following code gets the index of all the bedrooms that have bathrooms value
idx_checkbed = (df[df["checkbed"] == True]).index.tolist()

len(idx_checkbed)

In [ ]:
#it checks if the values of subset of  bedrooms and bathrooms that has been reversed are same 
if(idx_checkbath == idx_checkbed):
    print("Both list are same")

In [ ]:
# it swaps the values of subsets of bathrooms and bedrooms 
df.loc[idx_checkbath,['bathrooms','bedrooms']] = df.loc[idx_checkbath,['bedrooms','bathrooms']].values



In [ ]:
#Lets check if the values has been changed
df['checkbed'] = df['bedrooms'].apply(lambda value: value.startswith(' Number of bathrooms'))
len(df[df["checkbed"] == True])

The subset of bathroom and bedroom that was interchanged has been updated.

#### Checking the consistency of the area.sqft_living/sqft_lot

In [ ]:
# the following code checks if all the values in the colummn start with sqft living
df['check_sqftliving'] = df['area.sqft_living/sqft_lot'].apply(lambda value: value.startswith('sqft_living'))


df[ df['check_sqftliving'] == False ]

From the above output we can see that the column is consistent and ready to split

In [ ]:
# the following code splits the colums into two column with "=" and later splits the area 
# column into sqft_living and sqft_lot
df = df.join(df['area.sqft_living/sqft_lot'].str.split('=', 1, expand=True).rename(columns={0:'Reduntant', 1:'Area'}))

df = df.join(df['Area'].str.split('\\', 1, expand=True).rename(columns={0:'sqft_living', 1:'sqft_lot'}))

In [ ]:
# lets have a look into the dataframe and then delete the reduntant columns
df.head()



The column address, room, area, area.sqft_living/sqft_lot has been split and is no longer required.

The other column which were introduced to check the consistency are no longer required and hence should be dropped.

In [ ]:
del df['address']
del df['area.sqft_living/sqft_lot']
del df['Reduntant']
del df['check_sqftliving']
del df["Area"]
del df["rooms"]
del df['checkbed']
del df['checkbath']

In [ ]:
df.head()

Now extracting only the numbers out of the bedrooms and bathrooms column

In [ ]:
# reg exp has been taken from https://stackoverflow.com/questions/12475704/regular-expression-to-allow-only-integer-and-decimal
df['bedrooms'] = df['bedrooms'].str.extract(r"Number of bedrooms:\s(\d*[.]?\d*$)" , expand = True)

df['bathrooms'] = df['bathrooms'].str.extract(r"Number of bathrooms:\s(\d*[.]?\d*$)" , expand = True)

In [ ]:
df.head()

Now , Let's rename and rearrange the columns according to the given requirement

In [ ]:
df.rename(columns = {'area.sqft_above':'sqft_above', 'area.sqft_basement':'sqft_basement'}, inplace = True)

df = df.reindex(columns =['date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 
                            'view', 'condition', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'street', 
                            'city', 'statezip', 'country'])

In [ ]:
df.head()

#### Checking the data types of all the columns

In [ ]:
df.dtypes

After looking into the datatype of the column following column's datatypes should be changed.

1. **date** : should be changed to datetime64

2. **bedrooms , sqft_living, sqft_lot** : should be changed to int64

3. **bathroom**: should be changed to float64

In [ ]:
df['sqft_living'] = df['sqft_living'].astype('int64')
df['sqft_lot'] = df['sqft_lot'].astype('int64')
df['bedrooms'] = df['bedrooms'].astype('int64')
df['bathrooms'] = df['bathrooms'].astype('float64')
#df['street'] = df['street'].astype('string')
#df['city'] = df['city'].astype('string')
#df['street'] = df['street'].astype('string')
#df['statezip'] = df['statezip'].astype('string')
#df['country'] = df['country'].astype('string')

In [ ]:
df.dtypes

Now changing the data type of date column to datetime64

In [ ]:
#df['date'] = pd.to_datetime(df['date'], format="%Y%m%dT%H%M%S")

Running the above commented code to change the datatype of date column gives an error of "**day is out of range**"

Lets now define a function which validated the date and store the result in a new column

In [ ]:
#https://stackoverflow.com/questions/16870663/how-do-i-validate-a-date-string-format-in-python
# the following code defines a function to validatge the date
import datetime 
def validate(date_text):
    result = None
    try:
        datetime.datetime.strptime(date_text, "%Y%m%dT%H%M%S")
        result = 'NoError'
    except ValueError:
        result = 'Error'
    return result

In [ ]:
#del df['DateCheck']

In [ ]:
#the following code applies the validate function on every value of 
#date column and store the  value in DateCheck column
df['DateCheck'] = df['date'].apply(validate)

In [ ]:
#gives the date with error
df[df['DateCheck'] == 'Error']

From the above output we can see that the first error date has the value of 31 days for the month of june, which is not possible. This error should be a data entry error which would have occured either pressing 6 instead of 5 or 7, or this would have occurred entering 31 instead of 30. Lets assume the latter to be true , and we will update this date to 20140630T000000.

The second error date is due to the format issue, we will change it to the correct format matching with the other values.

In [ ]:
#getting the index of the improper date values
df[df['DateCheck'] == 'Error'].index

In [ ]:
# updating the dates
df = df.set_value(4334, 'date', '20140630T000000')
df = df.set_value(4335, 'date', '20140523T000000')

In [ ]:
#df.head()

In [ ]:
# deleting the check column
del df['DateCheck']

In [ ]:
#removing all the unnecessary spaces from the string columns
df['country'] = df['country'].str.strip()
df['statezip'] = df['statezip'].str.strip()
df['city'] = df['city'].str.strip()

In [ ]:
#df.head()

In [ ]:
#Now convberting the date column to datetime data type
df['date'] = pd.to_datetime(df['date'], format= "%Y%m%dT%H%M%S")

In [ ]:
df.head()

In [ ]:
df.dtypes

## Cleansing the file

In [ ]:
# Looking into the summary stats of the data
df.info()

From the information above, we found `yr_renovated` is the only column with missing values, which in fact can be possible because there would be some houses which has not been renovated at all.

In [ ]:
# Look at summary statistics of numeric data

df.describe()

The information above tell us the following: 

- `price` has the minimum value of zero. This looks suispicious and should be checked.

- `floors` is ranged from 1 to 3.5.

- `bathrooms` is ranged from 0 to 8. We need to check and  to make sure this column only contains numbers with `.0`, `.25`, `.5`, and `.75`.

Befor beginning with the data cleansing process , lets first check the duplicate records

In [ ]:
df[df.duplicated(keep = False)]

We see that there are two records with exacly the same information , with same price and same date. The record could be possible but looks suspicious as the same property cannot be sold more than once on a single day. Thus we assume that the second record is duplicate, and therefore we remove it.

In [ ]:
#removing the record at 4337 index
df.drop(4337, 0, inplace=True)

In [ ]:
#df.shape

#### Let's start looking into the column one by one

1) Inspeting **Country** column

In [ ]:
df['country'].describe()

In [ ]:
df["country"].unique()

There looks no issue in the country column as all the records are from USA

2) Inspecting **Statezip** column

In [ ]:
df["statezip"].describe()

In [ ]:
 sorted(df['statezip'].unique())

We found that all the values start WA followed by Zipcode. On searching the google we fould that all the zipcodes in the data are from WA(Washington) Reference: http://www.zipcodestogo.com/Washington/

3) Inspecting **City** column

In [ ]:
df["city"].describe()

In [ ]:
sorted(df['city'].unique())

ref: https://en.wikipedia.org/wiki/List_of_cities_in_Washington#List_of_cities

From the above output and making a google search we found the following errors/typos in the city column:

InCorrect City Name| Correct City Name
---------------|------------
Auburnt, auburn | Auburn
Belleview,Bellvue | Bellevue
Coronation|Carnation
Issaguah | Issaquah
Kirkllund | Kirkland
redmond,Redmonde,Redmund|Redmond
Samamish, sammamish |Sammamish
Seaattle, Seatle, seattlen|Seattle
Snogualmie | Snoqualmie
Woodenville | Woodinville|



In [ ]:
# Replace all incorrect cities with the correct name

df.replace({
        'Auburnt':'Auburn',
        'auburn':'Auburn',
        'Belleview':'Bellevue',
        'Bellvue':'Bellevue',
        'Coronation': 'Carnation',
        'Issaguah':'Issaquah',
        'Kirklund':'Kirkland',
        'Redmonde':'Redmond',
        'Redmund':'Redmond',
        'redmond':'Redmond',
        'Samamish':'Sammamish',
        'sammamish':'Sammamish',
        'Seaattle':'Seattle',
        'Seatle':'Seattle',
        'seattle':'Seattle',
        'Snogualmie':'Snoqualmie',
        'Sureline':'Shoreline',
        'Woodenville':'Woodinville'
    }, inplace=True)

In [ ]:
df[ df['city'] == 'auburn' ]

Inconsistencies and the typos has been fixed in the city column

4) Investigating **Street** column

In [ ]:
df['street'].describe()

There are 4525 unique values out of 4600. It could be because of the same street can be present in different cities and there is also a possiility of one property to be sold more than once. Lets check for the duplicates:

In [ ]:
df[df.duplicated(['street','statezip','yr_built','bedrooms','bathrooms','sqft_living','sqft_lot'], keep = False)].sort_values('street')

From the above output we can see that the inspite of having same address and same other information  the properties differ in  price and date. It is infact possible that the same property cna be sold multiple times on different day with different price.

5) Investigating **Year renovated** column

In [ ]:
df['yr_renovated'].describe()

There are many missing values ,because of the fact that the house may not have been renovated at all

6) Investigating **Yr built** column

In [ ]:
df['yr_built'].describe()

Everything looks fine with the yr_built column.

##### Checking the integrity constraint of the yr_built and yr_renovated
A property cannot be renovated before it is built

In [ ]:
df[ df['yr_renovated'] < df['yr_built'] ]

There are four records which violates the rule, possible reason could be the yr_built and yr_renovated values has been interchasnged. Lets change the values to correct year

In [ ]:
id = df[ df['yr_renovated'] < df['yr_built'] ].index.tolist()

df.loc[id,['yr_renovated','yr_built']] = df.loc[id,['yr_built','yr_renovated']].values

In [ ]:
df[ df['yr_renovated'] < df['yr_built'] ]

In [ ]:
df.loc[id]

The yr_built and yr_renovated integrity has been resolved and updated.

7) Investigating **View, Waterfront Floors and Condition** columns

In [ ]:
sorted( df['floors'].unique() )

In [ ]:
df['floors'].value_counts()

Looks everything fine with the floors

In [ ]:
sorted( df['waterfront'].unique() )

In [ ]:
df['waterfront'].value_counts()

Looks everything fine with the waterfront

In [ ]:
sorted( df['view'].unique() )

In [ ]:
df['view'].value_counts()

Looks everything fine with the view

In [ ]:
sorted( df['condition'].unique() )

In [ ]:
df['condition'].value_counts()

Looks everything fine with the condition

8) Investigating **Bedrooms and Bathrooms** columns

In [ ]:
sorted( df['bedrooms'].unique() )

In [ ]:
df['bedrooms'].value_counts()

There are 2 records with `0` bathrooms. Will check this again after investigating bedrooms column

In [ ]:
df['bedrooms'].describe()

In [ ]:
sorted( df['bathrooms'].unique() )

The values of bathrooms such as 1.05,2.3,2.55,2.57, etc are unaccepatable and should be rounded to nearest quarter.
There are few values with 0. Lets check them

In [ ]:
df['bathrooms'].value_counts()

There are two records of bathrooms with 0 values , will check them after converting all the values in the columm to the nearest quarter

In [ ]:
# ref:https://stackoverflow.com/questions/8118679/python-rounding-by-quarter-intervals
#the function round off a number to a nearest quarter
def roundPartial (value):
    return round (value / 0.25) * 0.25

In [ ]:
df['bathrooms'] = df['bathrooms'].apply(roundPartial)

In [ ]:
df['bathrooms'].value_counts()

The values of the bathrooms has been rounded off to the nearest quarter.

There are two cases where bathooms are 0 and two cases where bedrooms are 0.Lets check them if both of them occur tegether

In [ ]:
df[ (df['bedrooms'] == 0) & (df['bathrooms'] == 0) ]

When we check online , the bathrooms and bedrooms are not available online ref:  https://www.redfin.com/WA/Seattle/814-E-Howe-St-98102/home/2089103 and https://www.redfin.com/WA/Redmond/20418-NE-64th-Pl-98053/home/446188
 , so we will let these values as it is.

9) Investigating **sqft_above, sqft_basement, sqft_living, and  sqft_lot** columns

Two integrity rules should be met for these columns:

1) Sqft_living = sqft_basement + sqft_above

2) For a single floor building, sqft_lot > sqft_living

Checking the first integrity:

In [ ]:
df['checksqft'] = ( df['sqft_living'] == df['sqft_basement'] + df['sqft_above'] )

df[df['checksqft'] == False]

The above output shows that there are two records which doesnot meet the first integrity rule, 

Updating  these two records using the following code, which updates sqf_living = sqft_basement + sqft_above.

In [ ]:
inde = df[df['checksqft'] == False].index.tolist()

df.loc[inde,['sqft_living']] = df.loc[inde,['sqft_basement']].values + df.loc[inde,['sqft_above']].values 

#df[df['checksqft'] == False]
del df['checksqft']

##### Checking the second integrity rule

In [ ]:
# creates a new df with all the records of one floor property
df_floor = df[df['floors'] == 1]
#checks the integrity condition
df_floor['checklot'] = (df_floor['sqft_lot'] > df_floor['sqft_living'])

In [ ]:
df_floor[df_floor['checklot'] == False]

There is one record which violates the second integrity rule. Possible reason could be that the values has been interchanged while updating. To correct this error we interchange the values

In [ ]:
id = df_floor[df_floor['checklot'] == False].index.tolist()
id
df.loc[id,['sqft_lot','sqft_living']] = df.loc[id,['sqft_living','sqft_lot']].values

In [ ]:
df.loc[id]

The value has been correctly updated.

### Detecting outliers in the sqft_above, sqft_living, sqft_basement, sqft_lot

Getting summary stats of these variables

In [ ]:
df[['sqft_above', 'sqft_basement', 'sqft_living', 'sqft_lot']].describe()

In [ ]:
#import matplotlib.pyplot as plt

In [ ]:
#import seaborn as sb

We will investigate more about outliers by plotting boxplot of each colum. Lets start with the boxplot of sqft_above.

In [ ]:
sb.boxplot(df['sqft_above'])
plt.show()

In [ ]:
sqft_above_outliers = df[df.sqft_above > 4000]

In [ ]:
sqft_above_outliers['floors'].value_counts()

There are 108 records whose sqft_above is greater than 4000.It could be possible because many of the records are more than one floors

In [ ]:
sb.boxplot(df['sqft_lot'])
plt.show()

In [ ]:
sqft_lot_outlier = df[ df['sqft_lot'] > 400000 ]

In [ ]:
sqft_lot_outlier

There are possible 6 outliers, on checking with the internet(www.zillow.com) the rows at index 275,879,2480,3478 are correct as they match with the online source. The record at index 1078 is possibly correct as it contains a lot of address (16200- 16398).

The record at index 1539 doesnot match with the online source, we will change its to value to the one mentioned on the internet(https://www.zillow.com/homedetails/18923-SE-416th-St-Enumclaw-WA-98022/48760188_zpid/) 2.16 acres which equals 94090 sqft

In [ ]:
df = df.set_value(1539, 'sqft_lot', 94090)

In [ ]:
sb.boxplot(df['sqft_living'])
plt.show()

In [ ]:
sqft_living_outlier = df[ df['sqft_living'] > 8000 ]

In [ ]:
sqft_living_outlier

There are 8 outliers , all of which has 5 or more bedrooms and bigger sqft_lot and 80% has 2 or more floors. So the data looks fine.

In [ ]:
sb.boxplot(df['sqft_basement'])
plt.show()

In [ ]:
df[ df['sqft_basement'] > 4000 ]

On checking through the online sources the data looks fine, hence no error spotted.

### Investigating **Price** column

In [ ]:
df['price'].describe()

In [ ]:
price_null = df[df.price == 0]
price_null.shape

We can use linear regression  to predict the price for there 248 cases and then impute the predicted price in the final data set.
Use box plot to view outlier in the data.

In [ ]:
sb.boxplot(df['price'])
plt.show()

In [ ]:
#import numpy as np

In [ ]:
# Defining the IQR
#ref : http://stamfordresearch.com/outlier-removal-in-python-using-iqr-rule/

q75, q25 = np.percentile(df.price[df['price'] > 0], [75 ,25])
iqr = q75 - q25
 
minprice = q25 - (iqr*1.5)
maxprice = q75 + (iqr*1.5)

In [ ]:
#defining a new dataframe for the values of price greater than the
# 3 quarterile range
df_price = (df[df['price'] > maxprice]).sort_values('price', ascending = False)

In [ ]:
#getting the records for the price greater than max price and the sqft_lot
#less than 15000(mean of sqft lot) and sqft_living
#less than mean of sqft living
df_price[(df_price['sqft_lot'] < 15000) & (df_price['sqft_living'] < 2200)]

We have got 10 records as a potential outlier, Lets check first five of them with the online sources(www.zillow.com):

index|Our price| online price | WrongValue
-----|---------|-------------|------
4351| 26590000 | 265900| Yes
4347 | 12899000 | 490000|yes
4349|2199900 | --- | Maybe, because The data we have contains around 12 houses, but online sources shows the price for one house which is around 245K, so either we should update the whole record including all the column or  will completly remove this record. We will completly remove the record  for simplification. 
4348|2110000|211000|Yes
2767|1400000|1400000|No

In [ ]:
df = df.set_value(4351, 'price', 265900)
df = df.set_value(4347, 'price', 490000)
df = df.set_value(4348, 'price', 211000)
df.drop(4349, 0, inplace=True)

In [ ]:
df.loc[4351]

Prices has been updated  and now lets check the lower outliers.

In [ ]:
df[(df['price'] < 330000) & (df['price'] != 0)].sort_values('price', ascending = True)

We will check the first five values and check with nline source:

index|Our price| online price | WrongValue
-----|---------|-------------|------
4352| 7800 | 292000| Yes
1219 | 80000 | 230000|yes
1587|83000 | 83000 | N0 
4346|84350|1150000(estimated between 2013 n 2015|Yes
588|90000|247798|yes

In [ ]:
df = df.set_value(4352, 'price', 292000)
df = df.set_value(1219, 'price', 230000)
df = df.set_value(4346, 'price', 1150000)
df = df.set_value(588, 'price', 247798)

In [ ]:
df.loc[1219]

In [ ]:
df.head()

The price has been modified. The next step is to impute the prices for the records for which the price is zero, using Linear regression

### Imputing prices using linear regression

In [ ]:
#price_null

In [ ]:
#creating a copy of the data frame
df2 = df

In [ ]:
#creating a new column zip for the model 
df2['zip'] = df2['statezip'].str.extract(r"^WA (\d{5})")

In [ ]:
df2.head()

In [ ]:
df2['zip'] = df2['zip'].astype("int64")

In [ ]:
corr = df2.corr()
corr

We can see that sqft_living and sqft_above has a good correlation with price and the bedrooms view and sqft basement has fair co relation with price. Lets built a model using thses variabes.

In [ ]:
#taking the columns for the linear model
modl = df2[['sqft_living', 'sqft_above','bedrooms', 'view','sqft_basement','price']].copy()
modl = modl[ modl['price'] != 0 ]
modl = modl[ (modl['bedrooms'] != 0) ]


In [ ]:
# Create linear regression model

lm = LinearRegression()
# Fit the model
train_data = modl.iloc[:,:-1]
train_label = modl['price']
lm.fit(train_data,train_label)

# Calculate the predictive score
lm.score(train_data,train_label)

Around 55% variation in the dataset is explaied by the model. Lets try another model by converting few variables into categories

In [ ]:
#Converting few variables into categories
df2['zip'] = df['zip'].astype('category')
df2['yr_built'] = df['yr_built'].astype('category')
df2['bathrooms'] = df['bathrooms'].astype('category')
df2['bedrooms'] = df['bedrooms'].astype('category')
df2['condition'] = df['condition'].astype('category')
df2['view'] = df['view'].astype('category')
df2['waterfront'] = df['waterfront'].astype('category')
df2['floors'] = df['floors'].astype('category')

In [ ]:
#taking the columns for the linear model
model1 = df2[['sqft_living', 'sqft_above', 'floors', 'bathrooms', 
              'bedrooms', 'condition', 'view','waterfront','yr_built','zip',
              'price']].copy()
model1 = model1[ model1['price'] != 0 ]
model1 = model1[ (model1['bathrooms'] != 0) 
                  & (model1['bedrooms'] != 0) ]


In [ ]:
# Create linear regression model

lm = LinearRegression()
# Fit the model
train_data = model1.iloc[:,:-1]
train_label = model1['price']
lm.fit(train_data,train_label)

# Calculate the predictive score
lm.score(train_data,train_label)

The second model explains 60% variation in the data. So we will use the second model to impute the prices.

In [ ]:
#imputing the price on the main dataframe
df.loc[df['price'] == 0, 'price'] = lm.predict(df.loc[df['price'] == 0, ['sqft_living', 'sqft_above',
                                                                         'floors', 'bathrooms','bedrooms', 
                                                                         'condition', 'view','waterfront',
                                                                         'yr_built','zip']] )
# Round the price to 1 decimal places
df['price'] = df['price'].round(1)



In [ ]:
df[df["price"] == 0]

The above output shows the price data has been imputed.

In [ ]:
# deleting the column as it was not requird according to the task
del df['zip']

In [ ]:
#df.head()

In [ ]:
#df.dtypes

In [ ]:
#converting date to the given format
df['date'] = df['date'].dt.strftime('%Y%m%dT%H%M%S')
df2['yr_built'] = df['yr_built'].astype('int64')
df2['bathrooms'] = df['bathrooms'].astype('float64')
df2['bedrooms'] = df['bedrooms'].astype('int64')
df2['condition'] = df['condition'].astype('int64')
df2['view'] = df['view'].astype('int64')
df2['waterfront'] = df['waterfront'].astype('int64')
df2['floors'] = df['floors'].astype('float64')

In [ ]:
# Writing the data frame to a csv file
df.to_csv('ass2_data.csv', index=False)

## Summary

This assesment assessed the skills to process the raw data, clean it, parse it, make a tabular structure and  create a csv file from it. Skills gained after completing this assesment are:

* Getting familiar with reading and poarsing a JSON file into a dataframee.
* Gaining a thorough knowledge to use pandas library for the data frame manipulation.
* outlier detection with box blox using matplot and seaborn library.
* Building a regression model to impute a int column.
